In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
prop = pickle.load(open("../../datos/Data.p","rb"))

# Nos quedamos con la parte del set de datos que nos importa

In [3]:
p = prop[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]
p.dropna(inplace = True)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


# Separamos la Latitud-Longitud

In [4]:
p['lat'] = p.apply(lambda row: row[5][0], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
p['lon'] = p.apply(lambda row: row[5][1], axis=1)

/home/facundo/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
p = p[['property_type','place_name','surface_total_in_m2','surface_covered_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_LugaresTuristicos','Cant_ParadasTransporte','price_aprox_usd']]

# Asignamos un valor numerico al tipo de propiedad

In [7]:
def asignarPT( p ):
    if p == 'apartment':
        return 3
    if p == 'house':
        return 2.25
    if p == 'store':
        return 6.5
    return 2.5

In [8]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

# Asignamos un valor a los barrios

In [10]:
def asignarPV(name, dic):
    return dic[name]

In [11]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [12]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [13]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [14]:
p = p[['property_type','surface_total_in_m2',\
          'lat','lon','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value', 'price_aprox_usd']]

# Separamos los datos

In [15]:
Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'place_value']],p[['price_aprox_usd']],test_size=0.2)

# Hago algo similar con Decision Tree

In [19]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=3,min_samples_split=8, max_features = 'auto')
ada = AdaBoostRegressor(n_estimators = 100,base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

100725530206.132

In [20]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=3,min_samples_split=8, max_features = 'auto')
ada = AdaBoostRegressor(n_estimators = 110,base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

106931520418.42664

In [21]:
dt = DecisionTreeRegressor(criterion='mse',min_samples_leaf=3,min_samples_split=8, max_features = 'auto')
ada = AdaBoostRegressor(n_estimators = 90,base_estimator = dt)

ada.fit(Xtrn, Ytrn['price_aprox_usd'])
aux = Ytest.copy()
aux['predict'] = ada.predict(Xtest)

error = mean_squared_error(Ytest['price_aprox_usd'] , aux['predict'])
error

109797360333.45287